<a href="https://colab.research.google.com/github/soerenetler/squad-question-generation/blob/master/generate_datasets_4_human_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.system("git clone https://github.com/soerenetler/qg_with_attention.git")

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import ast
dataset = "squad"

# Human Data

In [ ]:
filepath = "/content/gdrive/MyDrive/mt-qg-data/01_data/preprocessedData/" + dataset + "/question_answer/test.csv"
human_df = pd.read_csv(filepath)[["question_text", "answer_sentence", "answer"]]

In [ ]:
human_df = human_df.sample(100)
human_df.to_csv("h_human_evaluation.csv")

# E2E Data

In [ ]:
data_filepath = "/content/gdrive/MyDrive/mt-qg-data/01_data/preprocessedData/" + dataset + "/question_answer/test.csv"
pred_filepath = "/content/gdrive/MyDrive/mt-qg-data/00_models/qg_attention/" + dataset + "/dataset_squad-target_length_20-input_length_40-vocab_input_45000-max_vocab_targ_28000-epochs_15-units_512-batch_64-layer_2-dropout_0.3-pretrained_True-bidirectional_True-answer_units_0-/test.txt"

e2e_df = pd.read_csv(filepath)[["answer_sentence", "answer"]]

pred_questions= []
with open(pred_filepath, "r") as f:
  for line in f:
    token = line.strip().split(" ")
    try:
      end_index = token.index("<end>")
    except ValueError:
      end_index = len(token)
    pred_questions.append(" ".join(token[:end_index]))


e2e_df["question_text"] = pred_questions
e2e_df = e2e_df.sample(100)

In [ ]:
e2e_df.head()

,answer_sentence,answer,question_text
6027,"As the Andes Mountains rose, however, a large ...",Solimões Basin,what was the lake lake ?
11152,Their competitors will take advantage of the s...,offering a higher wage the best of their labor,what do some employees assert to do ?
13224,"In simple terms, physical barriers prevent pat...",physical barriers,what is the term for rustication ?
11961,Founded by the American Baptist Education Soci...,William Rainey Harper,what was the first president of yale ?
5796,Free movement of goods within the European Uni...,a customs union,what is the result of the european union ?


In [ ]:
e2e_df.to_csv("e2e_human_evaluation.csv")

# Answer Separated Data

In [ ]:
data_filepath = "/content/gdrive/MyDrive/mt-qg-data/01_data/preprocessedData/" + dataset + "/question_answer/test.csv"
pred_filepath = "/content/gdrive/MyDrive/mt-qg-data/00_models/qg_attention/" + dataset + "/dataset_squad-target_length_20-input_length_40-vocab_input_45000-max_vocab_targ_28000-epochs_15-units_512-batch_64-layer_1-dropout_0.3-pretrained_True-bidirectional_True-answer_units_256-/test.txt"


ans_df = pd.read_csv(filepath)[["answer_sentence", "answer"]]

pred_questions= []
with open(pred_filepath, "r") as f:
  for line in f:
    token = line.strip().split(" ")
    try:
      end_index = token.index("<end>")
    except ValueError:
      end_index = len(token)
    pred_questions.append(" ".join(token[:end_index]))


ans_df["question_text"] = pred_questions

In [ ]:
ans_df = ans_df.sample(100)

In [ ]:
ans_df.to_csv("ans_human_evaluation.csv")

# Pipeline Data

In [ ]:
data_filepath = "/content/gdrive/MyDrive/mt-qg-data/01_data/preprocessedData/" + dataset + "/question_answer/trf_test.csv"
pred_filepath = "/content/gdrive/MyDrive/mt-qg-data/00_models/qg_attention/" + dataset + "/dataset_squad-target_length_20-input_length_40-vocab_input_45000-max_vocab_targ_28000-epochs_15-units_512-batch_64-layer_1-dropout_0.3-pretrained_True-bidirectional_True-answer_units_256-/trf_test.txt"


pipe_df = pd.read_csv(data_filepath)[["answer_sentence_token", "answer"]]
pipe_df["answer_sentence_token"]  = [ast.literal_eval(t) for t in pipe_df["answer_sentence_token"]]

pred_questions= []
with open(pred_filepath, "r") as f:
  for line in f:
    token = line.strip().split(" ")
    try:
      end_index = token.index("<end>")
    except ValueError:
      end_index = len(token)
    pred_questions.append(" ".join(token[:end_index]))

pipe_df["answer_sentence"] = [" ".join(t) for t in pipe_df["answer_sentence_token"]]
pipe_df.drop("answer_sentence_token", axis=1, inplace=True)
pipe_df["question_text"] = pred_questions
pipe_df = pipe_df.sample(100)

In [ ]:
pipe_df.head()

,answer,answer_sentence,question_text
2421,1852,Private schooling in the United States has bee...,in what year did the u.s. public school begin ?
3890,– whilst,A procedural consequence of the establishment ...,what is the irish parliament parliament ?
988,Air,"Air did not play a role in phlogiston theory ,...",what did feynman think was based on ?
1402,"200,000 Danish krone",UK law only required £ 1 of capital to start a...,what was the name of the company that was the ...
2890,– it,Undergraduate courses at the University of Chi...,what is the term for teaching at the universit...


In [ ]:
pipe_df.to_csv("pipe_human_evaluation.csv")